In [1]:
# get_regions dumps sa2_regions

In [30]:
import json, collections
import numpy

In [47]:
with open('../app/data/sa2_regions.json') as f:
    sa1_regions = json.load(f)

In [48]:
with open('../app/data/suburb_names.json') as f:
    suburbs = json.load(f)

In [49]:
with open('../app/data/sa1_to_suburb.json') as f:
    sa1_to_suburb = json.load(f)
suburb_to_sa1s = collections.defaultdict(list)
for sa1, suburb in sa1_to_suburb.items():
    suburb_to_sa1s[suburb].append(sa1)

In [50]:
sa1_to_sa1_regions = {r['sa2']:r for r in sa1_regions}

In [53]:
suburb_regions = []
for suburb_id, suburb_name in suburbs.items():
    constituents = [sa1_to_sa1_regions[sa1] for sa1 in suburb_to_sa1s[suburb_id]]
    if not constituents:
        continue
    d = dict(
        id=suburb_id,
        name=suburb_name,
    )
    meanable = 'lon', 'lat', 'religious', 'unemployment', 'rental_rate', 'median_rent', 'income'
    for m in meanable:
        d[m] = numpy.nanmean([c[m] for c in constituents])
    d['income_level'] = int(round(numpy.nanmean([c['income_level'] for c in constituents])))
    d['population'] = int(numpy.nansum([c['population'] for c in constituents]))
    d['sa1'] = suburb_to_sa1s[suburb_id]
    d['zoning'] = {k: 0 for k in 'RCIPWU'}
    d['businesses'] = list(numpy.mean([c['businesses'] for c in constituents], axis=0))
    for c in constituents:
        for z in d['zoning']:
            d['zoning'][z] += c['zoning'].get(z, 0) / len(constituents)
    suburb_regions.append(d)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice
  if sys.path[0] == '':


In [46]:
with open('../app/data/suburb_regions.json', 'w') as f:
    json.dump(suburb_regions, f)